### Word Embedding Manual with Creating token_type_ids

In [ ]:
from transformers import BertTokenizer
import torch

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input dengan lebih dari dua segmen
segments = [
    "What is photosynthesis?",  # Segmen 1
    "Photosynthesis is the process by which plants convert sunlight into energy.",  # Segmen 2
    "It occurs in the chloroplasts of plant cells."  # Segmen 3
]

# Encode masing-masing segmen
encoded_segments = [tokenizer.encode(seg, add_special_tokens=False) for seg in segments]

# Gabungkan segmen dengan [SEP] di antaranya
input_ids = [tokenizer.cls_token_id]  # [CLS]
token_type_ids = []  # Untuk menyimpan ID tipe token
current_segment_id = 0

for segment in encoded_segments:
    input_ids.extend(segment + [tokenizer.sep_token_id])  # Tambahkan segmen dan [SEP]
    token_type_ids.extend([current_segment_id] * (len(segment) + 1))  # Token Type IDs
    current_segment_id += 1  # Pindah ke segmen berikutnya

# Padding untuk mencapai panjang maksimum
max_length = 50
attention_mask = [1] * len(input_ids)  # Mask untuk token yang relevan

# Tambahkan padding jika diperlukan
while len(input_ids) < max_length:
    input_ids.append(0)  # Token PAD
    attention_mask.append(0)
    token_type_ids.append(0)  # Token Type ID untuk padding

# Pastikan panjangnya sesuai
input_ids = input_ids[:max_length]
attention_mask = attention_mask[:max_length]
token_type_ids = token_type_ids[:max_length]

# Konversi ke tensor PyTorch
input_ids = torch.tensor([input_ids])
attention_mask = torch.tensor([attention_mask])
token_type_ids = torch.tensor([token_type_ids])

# Output
print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)
print("Token Type IDs:", token_type_ids)


# Adding custom embedding to handle more than 2 segment in BERT model varian

In [ ]:
import torch
import torch.nn as nn
from transformers import AlbertModel

class CustomAlbertModel(nn.Module):
    def __init__(self, model_name='indobenchmark/indobert-lite-base-p2', num_token_types=3):
        super().__init__()
        # Load ALBERT model
        self.albert = AlbertModel.from_pretrained(model_name)
        
        # Replace token_type_embeddings to support more token types
        self.albert.embeddings.token_type_embeddings = nn.Embedding(
            num_embeddings=num_token_types, 
            embedding_dim=self.albert.config.hidden_size
        )
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Pass through modified ALBERT model
        return self.albert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids
        )

# Create model instance
model = CustomAlbertModel(num_token_types=3)

# Test model
input_ids = torch.randint(0, 30000, (4, 512))  # Example input
attention_mask = torch.ones(4, 512)  # Example mask
token_type_ids = torch.randint(0, 3, (4, 512))  # Example token types (0, 1, 2)

output = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
print(output.last_hidden_state.shape)  # Should be (batch_size, sequence_length, hidden_size)


# Multi-Task Learning

In [ ]:
class MultiTaskModel(nn.Module):
    def __init__(self, bert_model_name):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.task1_head = nn.Linear(self.bert.config.hidden_size, 1)  # Output for Task 1
        self.task2_head = nn.Linear(self.bert.config.hidden_size, 1)  # Output for Task 2

    def forward(self, input_ids, attention_mask, token_type_ids, task):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        if task == 1:
            return self.task1_head(cls_embedding)
        elif task == 2:
            return self.task2_head(cls_embedding)

for epoch in range(num_epochs):
    for batch_task1, batch_task2 in zip(loader_task1, loader_task2):
        # Task 1
        optimizer.zero_grad()
        output1 = model(batch_task1['input_ids'], batch_task1['attention_mask'], task=1)
        loss1 = criterion(output1, batch_task1['scores'])
        loss1.backward()
        optimizer.step()

        # Task 2
        optimizer.zero_grad()
        output2 = model(batch_task2['input_ids'], batch_task2['attention_mask'], task=2)
        loss2 = criterion(output2, batch_task2['scores'])
        loss2.backward()
        optimizer.step()


# Chunk without separating segment

In [ ]:
from transformers import BertTokenizer
import torch
torch.set_printoptions(threshold=torch.inf)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-lite-base-p2')

question = "Apa yang anda ketahui mengenai kerajaan Majapahit? (Mengenai tempat, masa puncak kejayaan dan agama. Usahakan jawab dalam 2-4 kalimat)"
ref_answer = "Majapahit adalah sebuah kerajaan yang berpusat di Jawa Timur, Indonesia. Kerajaan ini mencapai puncak kejayaannya dan menguasai wilayah di Nusantara pada masa kekuasaan Hayam Wuruk. Kerajaan Majapahit merupakan kerajaan Hindu - Budha terbesar dalam sejarah Indonesia."
answer = "Majapahit adalah sebuah kerajaan yang berpusat di Jawa Timur, Indonesia, yang pernah berdiri dari sekitar tahun 1293 hingga 1500 M. Kerajaan ini mencapai puncak kejayaannya menjadi kemaharajaan raya yang menguasai wilayah yang luas di Nusantara pada masa kekuasaan Hayam Wuruk, yang berkuasa dari tahun 1350 hingga 1389. Kerajaan Majapahit adalah kerajaan Hindu-Buddha terakhir yang menguasai Nusantara dan dianggap sebagai salah satu dari negara terbesar dalam sejarah Indonesia.[2] Menurut Negarakertagama, kekuasaannya terbentang di Jawa, Sumatra, Semenanjung Malaya, Kalimantan, hingga Indonesia timur, meskipun wilayah kekuasaannya masih diperdebatkan. Sebelum berdirinya Majapahit, Singhasari telah menjadi kerajaan paling kuat di Jawa. Hal ini menjadi perhatian Kubilai Khan, penguasa Dinasti Yuan di Tiongkok. Ia mengirim utusan yang bernama Meng Chi[14] ke Singhasari yang menuntut upeti. Kertanagara, penguasa kerajaan Singhasari yang terakhir menolak untuk membayar upeti dan mempermalukan utusan tersebut dengan merusak wajahnya dan memotong telinganya.[14][15] Kubilai Khan marah dan lalu memberangkatkan ekspedisi besar ke Jawa tahun 1293. Ketika itu, Jayakatwang, adipati Kediri, sudah menggulingkan dan membunuh Kertanegara. Atas saran Aria Wiraraja, Jayakatwang memberikan pengampunan kepada Raden Wijaya, menantu Kertanegara, yang datang menyerahkan diri. Kemudian, Wiraraja mengirim utusan ke Daha, yang membawa surat berisi pernyataan, Raden Wijaya menyerah dan ingin mengabdi kepada Jayakatwang.[16] Jawaban dari surat di atas disambut dengan senang hati.[16] Raden Wijaya kemudian diberi hutan Tarik. Ia membuka hutan itu dan membangun desa baru. Desa itu dinamai Majapahit, yang namanya diambil dari buah maja, dan rasa 'pahit' dari buah tersebut. Ketika pasukan Mongol tiba, Wijaya bersekutu dengan pasukan Mongol untuk bertempur melawan Jayakatwang. Setelah berhasil menjatuhkan Jayakatwang, Raden Wijaya berbalik menyerang sekutu Mongolnya sehingga memaksa mereka menarik pulang kembali pasukannya secara kalang-kabut karena mereka berada di negeri asing.[17][18] Saat itu juga merupakan kesempatan terakhir mereka untuk menangkap angin muson agar dapat pulang, atau mereka terpaksa harus menunggu enam bulan lagi di pulau yang asing. Tanggal pasti yang digunakan sebagai tanggal kelahiran kerajaan Majapahit adalah hari penobatan Raden Wijaya sebagai raja, yaitu tanggal 15 bulan Kartika tahun 1215 saka yang bertepatan dengan tanggal 10 November 1293. Ia dinobatkan dengan nama resmi Kertarajasa Jayawardhana. Kerajaan ini menghadapi masalah. Beberapa orang terpercaya Kertarajasa, termasuk Ranggalawe, Sora, dan Nambi memberontak melawannya, meskipun pemberontakan tersebut tidak berhasil. Pemberontakan Ranggalawe ini didukung oleh Panji Mahajaya, Ra Arya Sidi, Ra Jaran Waha, Ra Lintang, Ra Tosan, Ra Gelatik, dan Ra Tati. Semua ini tersebut disebutkan dalam Pararaton.[19] Slamet Muljana menduga bahwa mahapatih Halayudha lah yang melakukan konspirasi untuk menjatuhkan semua orang tepercaya raja, agar ia dapat mencapai posisi tertinggi dalam pemerintahan. Namun setelah kematian pemberontak terakhir (Kuti), Halayudha ditangkap dan dipenjara, dan lalu dihukum mati.[18] Wijaya meninggal dunia pada tahun 1309. Putra dan penerus Wijaya adalah Jayanegara. Pararaton menyebutnya Kala Gemet, yang berarti 'penjahat lemah'. Kira-kira pada suatu waktu dalam kurun pemerintahan Jayanegara, seorang pendeta Italia, Odorico da Pordenone mengunjungi keraton Majapahit di Jawa. Pada tahun 1328, Jayanegara dibunuh oleh tabibnya, Tanca. Ibu tirinya yaitu Gayatri Rajapatni seharusnya menggantikannya, akan tetapi Rajapatni memilih mengundurkan diri dari istana dan menjadi bhiksuni. Rajapatni menunjuk anak perempuannya Tribhuwana Wijayatunggadewi untuk menjadi ratu Majapahit. Pada tahun 1336, Tribhuwana menunjuk Gajah Mada sebagai Mahapatih, pada saat pelantikannya Gajah Mada mengucapkan Sumpah Palapa yang menunjukkan rencananya untuk melebarkan kekuasaan Majapahit dan membangun sebuah kemaharajaan. Selama kekuasaan Tribhuwana, kerajaan Majapahit berkembang menjadi lebih besar dan terkenal di kepulauan Nusantara. Tribhuwana berkuasa di Majapahit sampai kematian ibunya pada tahun 1350. Ia diteruskan oleh putranya, Hayam Wuruk."
text = f"Question: {question} Reference Answer: {ref_answer} [SEP] Student Answer: {answer}"

tokens = tokenizer.encode_plus(text, add_special_tokens=False, truncation=False, return_tensors='pt')

token_type_ids = []
current_token = 0
for token in tokens['input_ids'].flatten():
    if(token == 0):
        token_type_ids.append(0)
        continue
    token_type_ids.append(current_token)
    if(token == 102 or token == 3): # 102 is token SEP for bert-base and 3 is for albert-lite
        current_token += 1

input_ids = tokens['input_ids'].flatten()
attention_mask = tokens['attention_mask'].flatten()
token_type_ids = torch.tensor(token_type_ids)
token_type_ids

# chunking hierarkikal -> no overlapping
chunks = []
max_len = 512
cls_token = torch.tensor([2]) # 101 untuk bert, 2 untuk indobert
sep_token = torch.tensor([3]) # 102 untuk bert, 3 untuk indobert

for i in range(0, len(input_ids), max_len):
    chunk_input_ids = input_ids[i: i+max_len]
    chunk_att_mask = attention_mask[i: i+max_len]
    chunk_token_type = token_type_ids[i: i+max_len]

    # menentukan segmen yang benar untuk token [CLS] dan [SEP] yang baru ditambahkan -> TANYA GPT LAGI NANTI
    first_token_type = chunk_token_type[0].unsqueeze(0) 
    last_token_type = chunk_token_type[-1].unsqueeze(0) 

    # menambahkan token [CLS] diawal teks saja
    if(i == 0):
        chunk_input_ids = torch.cat([cls_token, chunk_input_ids])
        chunk_att_mask = torch.cat([torch.ones(1, dtype=torch.long), chunk_att_mask])
        chunk_token_type = torch.cat([first_token_type, chunk_token_type])
    else:
        # chunk sisanya tambahkan [SEP] diawal
        chunk_input_ids = torch.cat([sep_token, chunk_input_ids])
        chunk_att_mask = torch.cat([torch.ones(1, dtype=torch.long), chunk_att_mask])
        chunk_token_type = torch.cat([first_token_type, chunk_token_type])

    # Tambahkan [SEP] di akhir tiap chunk
    chunk_input_ids = torch.cat([chunk_input_ids, sep_token])
    chunk_att_mask = torch.cat([chunk_att_mask, torch.ones(1, dtype=torch.long)])
    chunk_token_type = torch.cat([chunk_token_type, last_token_type])

    # menambahkan padding pada chunk terakhir agar memastikan panjang tiap chunk itu sama
    if len(chunk_input_ids) < max_len:
        padding_length = max_len - len(chunk_input_ids)

        # assign padding 0
        chunk_input_ids = torch.cat([chunk_input_ids, torch.zeros(padding_length, dtype=torch.long)])
        chunk_att_mask = torch.cat([chunk_att_mask, torch.zeros(padding_length, dtype=torch.long)])
        chunk_token_type = torch.cat([chunk_token_type, torch.zeros(padding_length, dtype=torch.long)])

    result = {
        'input_ids': chunk_input_ids,
        'attention_mask': chunk_att_mask,
        'token_type_ids': chunk_token_type,
    }

    chunks.append(result)

chunks
# for item in chunks:
#     print(len(item['input_ids']))

# CHUNK WITH SEGMENT SEPARATION

In [ ]:
question = "Apa yang anda ketahui mengenai kerajaan Majapahit? (Mengenai tempat, masa puncak kejayaan dan agama. Usahakan jawab dalam 2-4 kalimat)"
ref_answer = "Majapahit adalah sebuah kerajaan yang berpusat di Jawa Timur, Indonesia. Kerajaan ini mencapai puncak kejayaannya dan menguasai wilayah di Nusantara pada masa kekuasaan Hayam Wuruk. Kerajaan Majapahit merupakan kerajaan Hindu - Budha terbesar dalam sejarah Indonesia."
answer = "Majapahit adalah sebuah kerajaan yang berpusat di Jawa Timur, Indonesia, yang pernah berdiri dari sekitar tahun 1293 hingga 1500 M. Kerajaan ini mencapai puncak kejayaannya menjadi kemaharajaan raya yang menguasai wilayah yang luas di Nusantara pada masa kekuasaan Hayam Wuruk, yang berkuasa dari tahun 1350 hingga 1389. Kerajaan Majapahit adalah kerajaan Hindu-Buddha terakhir yang menguasai Nusantara dan dianggap sebagai salah satu dari negara terbesar dalam sejarah Indonesia.[2] Menurut Negarakertagama, kekuasaannya terbentang di Jawa, Sumatra, Semenanjung Malaya, Kalimantan, hingga Indonesia timur, meskipun wilayah kekuasaannya masih diperdebatkan. Sebelum berdirinya Majapahit, Singhasari telah menjadi kerajaan paling kuat di Jawa. Hal ini menjadi perhatian Kubilai Khan, penguasa Dinasti Yuan di Tiongkok. Ia mengirim utusan yang bernama Meng Chi[14] ke Singhasari yang menuntut upeti. Kertanagara, penguasa kerajaan Singhasari yang terakhir menolak untuk membayar upeti dan mempermalukan utusan tersebut dengan merusak wajahnya dan memotong telinganya.[14][15] Kubilai Khan marah dan lalu memberangkatkan ekspedisi besar ke Jawa tahun 1293. Ketika itu, Jayakatwang, adipati Kediri, sudah menggulingkan dan membunuh Kertanegara. Atas saran Aria Wiraraja, Jayakatwang memberikan pengampunan kepada Raden Wijaya, menantu Kertanegara, yang datang menyerahkan diri. Kemudian, Wiraraja mengirim utusan ke Daha, yang membawa surat berisi pernyataan, Raden Wijaya menyerah dan ingin mengabdi kepada Jayakatwang.[16] Jawaban dari surat di atas disambut dengan senang hati.[16] Raden Wijaya kemudian diberi hutan Tarik. Ia membuka hutan itu dan membangun desa baru. Desa itu dinamai Majapahit, yang namanya diambil dari buah maja, dan rasa 'pahit' dari buah tersebut. Ketika pasukan Mongol tiba, Wijaya bersekutu dengan pasukan Mongol untuk bertempur melawan Jayakatwang. Setelah berhasil menjatuhkan Jayakatwang, Raden Wijaya berbalik menyerang sekutu Mongolnya sehingga memaksa mereka menarik pulang kembali pasukannya secara kalang-kabut karena mereka berada di negeri asing.[17][18] Saat itu juga merupakan kesempatan terakhir mereka untuk menangkap angin muson agar dapat pulang, atau mereka terpaksa harus menunggu enam bulan lagi di pulau yang asing. Tanggal pasti yang digunakan sebagai tanggal kelahiran kerajaan Majapahit adalah hari penobatan Raden Wijaya sebagai raja, yaitu tanggal 15 bulan Kartika tahun 1215 saka yang bertepatan dengan tanggal 10 November 1293. Ia dinobatkan dengan nama resmi Kertarajasa Jayawardhana. Kerajaan ini menghadapi masalah. Beberapa orang terpercaya Kertarajasa, termasuk Ranggalawe, Sora, dan Nambi memberontak melawannya, meskipun pemberontakan tersebut tidak berhasil. Pemberontakan Ranggalawe ini didukung oleh Panji Mahajaya, Ra Arya Sidi, Ra Jaran Waha, Ra Lintang, Ra Tosan, Ra Gelatik, dan Ra Tati. Semua ini tersebut disebutkan dalam Pararaton.[19] Slamet Muljana menduga bahwa mahapatih Halayudha lah yang melakukan konspirasi untuk menjatuhkan semua orang tepercaya raja, agar ia dapat mencapai posisi tertinggi dalam pemerintahan. Namun setelah kematian pemberontak terakhir (Kuti), Halayudha ditangkap dan dipenjara, dan lalu dihukum mati.[18] Wijaya meninggal dunia pada tahun 1309. Putra dan penerus Wijaya adalah Jayanegara. Pararaton menyebutnya Kala Gemet, yang berarti 'penjahat lemah'. Kira-kira pada suatu waktu dalam kurun pemerintahan Jayanegara, seorang pendeta Italia, Odorico da Pordenone mengunjungi keraton Majapahit di Jawa. Pada tahun 1328, Jayanegara dibunuh oleh tabibnya, Tanca. Ibu tirinya yaitu Gayatri Rajapatni seharusnya menggantikannya, akan tetapi Rajapatni memilih mengundurkan diri dari istana dan menjadi bhiksuni. Rajapatni menunjuk anak perempuannya Tribhuwana Wijayatunggadewi untuk menjadi ratu Majapahit. Pada tahun 1336, Tribhuwana menunjuk Gajah Mada sebagai Mahapatih, pada saat pelantikannya Gajah Mada mengucapkan Sumpah Palapa yang menunjukkan rencananya untuk melebarkan kekuasaan Majapahit dan membangun sebuah kemaharajaan. Selama kekuasaan Tribhuwana, kerajaan Majapahit berkembang menjadi lebih besar dan terkenal di kepulauan Nusantara. Tribhuwana berkuasa di Majapahit sampai kematian ibunya pada tahun 1350. Ia diteruskan oleh putranya, Hayam Wuruk."
text1 = f"Question: {question} Reference Answer: {ref_answer}"
text2 = f"Student Answer: {answer}"

tokens1 = tokenizer.encode_plus(text1, add_special_tokens=False, truncation=False, return_tensors='pt')
tokens2 = tokenizer.encode_plus(text2, add_special_tokens=False, truncation=False, return_tensors='pt')

def create_token_type(input_ids, segment_num):
    token_type_ids = []
    for token in input_ids:
        if token == 0:
            token_type_ids.append(0)
            continue
        token_type_ids.append(segment_num)
    return torch.tensor(token_type_ids)

# SEGMENT 1
input_ids1 = tokens1['input_ids'].flatten()
attention_mask1 = tokens1['attention_mask'].flatten()
token_type_ids1 = create_token_type(input_ids1, 0)

# SEGMENT 2
input_ids2 = tokens2['input_ids'].flatten()
attention_mask2 = tokens2['attention_mask'].flatten()
token_type_ids2 = create_token_type(input_ids2, 1)

# CREATE CHUNK WITHOUT OVERLAPPING -> HIERARKIKAL
chunks = []
max_len = 510
cls_token = torch.tensor([2])
sep_token = torch.tensor([3])

def create_chunks(input_ids, attention_mask, token_type_ids, segment_num, stride=max_len):
    chunk = []
    for i in range(0, len(input_ids), stride):
        print(i, i+max_len)
        flag_padding = 0
        chunk_input_ids = input_ids[i: i+max_len]
        chunk_att_mask = attention_mask[i: i+max_len]
        chunk_token_type = token_type_ids[i: i+max_len]

        # Tambahkan token [CLS] khusus untuk segment dan chunk pertama 
        if(segment_num == 0 and i == 0):
            chunk_input_ids = torch.cat([cls_token, chunk_input_ids])
            chunk_att_mask = torch.cat([torch.ones(1, dtype=torch.long), chunk_att_mask])
            chunk_token_type = torch.cat([torch.tensor([segment_num]), chunk_token_type])

            # Tambahkan [SEP] di akhir tiap chunk
            chunk_input_ids = torch.cat([chunk_input_ids, sep_token])
            chunk_att_mask = torch.cat([chunk_att_mask, torch.ones(1, dtype=torch.long)])
            chunk_token_type = torch.cat([chunk_token_type, torch.tensor([segment_num])])
        else:
            # Add token [SEP] di awal chunk seterusnya
            chunk_input_ids = torch.cat([sep_token, chunk_input_ids])
            chunk_att_mask = torch.cat([torch.ones(1, dtype=torch.long), chunk_att_mask])
            chunk_token_type = torch.cat([torch.tensor([segment_num]), chunk_token_type])

            # Tambahkan [SEP] di akhir tiap chunk
            chunk_input_ids = torch.cat([chunk_input_ids, sep_token])
            chunk_att_mask = torch.cat([chunk_att_mask, torch.ones(1, dtype=torch.long)])
            chunk_token_type = torch.cat([chunk_token_type, torch.tensor([segment_num])])

        # menambahkan padding pada chunk terakhir agar memastikan panjang tiap chunk itu sama
        if len(chunk_input_ids) < max_len:
            flag_padding = 1
            padding_length = max_len - len(chunk_input_ids)

            # assign padding 0
            chunk_input_ids = torch.cat([chunk_input_ids, torch.zeros(padding_length, dtype=torch.long)])
            chunk_att_mask = torch.cat([chunk_att_mask, torch.zeros(padding_length, dtype=torch.long)])
            chunk_token_type = torch.cat([chunk_token_type, torch.zeros(padding_length, dtype=torch.long)])

        chunk.append({
            'input_ids': chunk_input_ids,
            'attention_mask': chunk_att_mask,
            'token_type_ids': chunk_token_type
        })

        if flag_padding == 1:
            # stop stride chunk
            break

    return chunk

# WITHOUT OVERLAPPING
chunks_segment1 = create_chunks(input_ids1, attention_mask1, token_type_ids1, segment_num=0)
chunks_segment2 = create_chunks(input_ids2, attention_mask2, token_type_ids2, segment_num=1)

# WITH OVERLAPPING
chunks_segment1 = create_chunks(input_ids1, attention_mask1, token_type_ids1, segment_num=0, stride=max_len-128)
chunks_segment2 = create_chunks(input_ids2, attention_mask2, token_type_ids2, segment_num=1, stride=max_len-128)

# Collate FN for dataloader

In [ ]:
def custom_collate_fn(batch):
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []
    all_scores = []

    for chunks, score in batch:
        for chunk in chunks:
            all_input_ids.append(chunk['input_ids'])
            all_attention_mask.append(chunk['attention_mask'])
            all_token_type_ids.append(chunk['token_type_ids'])
            all_scores.append(score)

    return {
        'input_ids': torch.stack(all_input_ids),
        'attention_mask': torch.stack(all_attention_mask),
        'token_type_ids': torch.stack(all_token_type_ids),
        'scores': torch.tensor(all_scores, dtype=torch.float)
    }

In [ ]:
def collate_fn(batch):
    """
    Fungsi collate untuk menangani batch data dengan beberapa chunk per sample dan memisahkan data dan label.
    """
    all_chunks = []
    all_labels = []

    # Iterasi setiap sample dalam batch
    for sample in batch:
        # Setiap sample adalah tuple (data, label), kita pisahkan
        data, label = sample
        
        # Data berisi list of dict (chunk-chunk untuk sample ini)
        all_chunks.append(data)  # Menambahkan list of dict (chunks) ke all_chunks
        all_labels.append(label)  # Menambahkan label ke all_labels

    # Menggabungkan label menjadi tensor
    all_labels_tensor = torch.stack(all_labels)  # Stack menjadi tensor (batch_size, 1)

    return all_chunks, all_labels_tensor  # Mengembalikan batch data (chunks) dan labels

# Attention Pooling with sum pooling and mask self-attention diagonal untuk menghindari self-attention (chunk memperhatikan dirinya sendiri)

In [ ]:
def attention_pooling(self, chunk_outputs):
    """Menggabungkan output chunks menggunakan attention pooling dengan normalisasi."""
    # Normalisasi chunk embeddings
    stacked_chunks = torch.nn.functional.normalize(torch.cat(chunk_outputs, dim=0), dim=1)  # [num_chunks, hidden_size]

    # Hitung bobot perhatian
    attention_scores = stacked_chunks @ stacked_chunks.T
    attention_scores.fill_diagonal_(-float('inf'))  # Mask self-attention diagonal
    attention_weights = torch.nn.functional.softmax(attention_scores, dim=1)

    # Pooling dengan perhatian
    pooled_output = attention_weights @ stacked_chunks

    # Weighted sum pooling (mengganti mean pooling)
    return torch.sum(pooled_output, dim=0)  # [hidden_size]